# Cryptocurrency Recommendation System Analysis
### Authors


|    Student Name                 |    Student Number  |
|---------------------------------|--------------------|
| Raj Sandhu                      | 101111960          |
| Akaash Kapoor                   | 101112895          |
| Ali Alvi                        | 101114940          |
| Hassan Jallad                   | 101109334          |
| Areeb Ul Haq                    | 101115337          |
| Ahmad Abuoudeh                  | 101072636          |

## Libraries To Install

In [1]:
! pip install kneed
# installs kneed library which is used to determine the knee point to be used for recommendation systems.

## Libraries to Import

In [2]:
import json
import pandas as pd
import os
from kneed import KneeLocator

## Read In Coin Similarity Matrices and Processed Coin Dataset

In [3]:
parent_folder = os.path.dirname(os.path.dirname(os.getcwd())) #Parent folder of the repo
model_folder = "models" #Name of folder that the similarity matrices are stored.
data_folder = "data" #Name of folder that the coin data it stored.
proc_folder = "processed" #Name of folder that the processed coin dataset is stored.
model_data_file_path = os.path.join(parent_folder, model_folder) #path to models folder
processed_data_file_path = os.path.join(parent_folder, data_folder, proc_folder) #Path to processed coin dataset.

#load in data from cvs files
coin_df_wmd = pd.read_csv(open(os.path.join(model_data_file_path, "coin-similarity-matrix-description.csv"), "r"), index_col="Name")
coin_df_ed = pd.read_csv(open(os.path.join(model_data_file_path, "coin-similarity-matrix-euclidean-distance.csv"), "r"), index_col="Name")
coin_df_weighted_avg = pd.read_csv(open(os.path.join(model_data_file_path, "coin-similarity-matrix-weighted-average.csv"), "r"), index_col="Name")
processed_coin_df = pd.read_csv(open(os.path.join(processed_data_file_path, "coin-info.csv"), "r"))

#test output of the wmd similarity matrix.
coin_df_wmd

,iota,anchor-protocol,compound,bitcoin-sv,drep,moonbeam,usd-coin,chainlink,basic-attention-token,bittorrent,...,gala,bitcoin-gold,render-token,unfoldu-group-coin-(new),maker,nexus-mutual,juno,okb,avalanche,compound-usd-coin
Name,,,,,,,,,,,,,,,,,,,,,
iota,0.000000,3.170357,2.171160,2.229937,2.735465,2.258385,2.208431,2.103930,2.225857,2.376386,...,2.772715,2.334604,2.705839,2.391945,2.102176,2.190762,2.375256,2.178784,2.516597,2.509003
anchor-protocol,3.170357,0.000000,3.326736,3.277865,3.407600,3.268028,3.369509,3.340737,3.381832,3.322350,...,3.450060,3.292741,3.360755,3.208156,3.191857,3.253310,3.346427,3.152493,3.380975,3.372036
compound,2.171160,3.326736,0.000000,2.281674,2.891091,2.400574,2.188473,2.137912,2.133458,2.439778,...,2.925447,2.358951,2.770554,2.393285,1.800100,2.407950,2.410279,2.302774,2.489660,2.248290
bitcoin-sv,2.229937,3.277865,2.281674,0.000000,2.843226,2.393085,2.277789,2.277939,2.292242,2.510614,...,2.917513,2.098307,2.752338,2.477368,2.233188,2.465231,2.507217,2.308496,2.574286,2.623275
drep,2.735465,3.407600,2.891091,2.843226,0.000000,2.810384,2.868590,2.770253,2.926254,2.970365,...,2.998832,2.827450,3.008179,2.844016,2.881863,2.844083,2.840785,2.764615,2.836954,3.020076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nexus-mutual,2.190762,3.253310,2.407950,2.465231,2.844083,2.458540,2.456764,2.374419,2.428557,2.453446,...,2.867152,2.513954,2.761209,2.371646,2.399128,0.000000,2.518541,2.237765,2.583840,2.703363
juno,2.375256,3.346427,2.410279,2.507217,2.840785,2.363450,2.412082,2.239042,2.504455,2.621340,...,3.086009,2.510507,2.909125,2.545965,2.367397,2.518541,0.000000,2.388843,2.540172,2.738081
okb,2.178784,3.152493,2.302774,2.308496,2.764615,2.014560,2.286681,2.299715,2.352729,2.253653,...,2.818819,2.235299,2.675767,2.259754,2.232096,2.237765,2.388843,0.000000,2.387466,2.515510


In [4]:
#Test output of the Euclidean Distance similarity matrix
coin_df_ed

,iota,anchor-protocol,compound,bitcoin-sv,drep,moonbeam,usd-coin,chainlink,basic-attention-token,bittorrent,...,gala,bitcoin-gold,render-token,unfoldu-group-coin-(new),maker,nexus-mutual,juno,okb,avalanche,compound-usd-coin
Name,,,,,,,,,,,,,,,,,,,,,
iota,0.000000,0.000468,0.094384,0.039394,0.000059,0.001550,2.368410e-04,0.004297,0.000071,2.371535e-04,...,0.000132,0.014342,0.000775,2.362516e-04,0.477463,0.017720,0.005296,0.003586,0.020100,0.000237
anchor-protocol,0.000468,0.000000,0.093916,0.038926,0.000409,0.001082,7.048557e-04,0.003829,0.000539,7.051683e-04,...,0.000600,0.013874,0.000307,7.042663e-04,0.476995,0.017252,0.004828,0.003118,0.019632,0.000705
compound,0.094384,0.093916,0.000000,0.054990,0.094325,0.092834,9.462082e-02,0.090087,0.094455,9.462113e-02,...,0.094516,0.080042,0.093609,9.462023e-02,0.383079,0.076663,0.089088,0.090798,0.074284,0.094621
bitcoin-sv,0.039394,0.038926,0.054990,0.000000,0.039335,0.037844,3.963052e-02,0.035097,0.039465,3.963083e-02,...,0.039526,0.025052,0.038619,3.962993e-02,0.438069,0.021673,0.034098,0.035808,0.019294,0.039630
drep,0.000059,0.000409,0.094325,0.039335,0.000000,0.001491,2.958295e-04,0.004238,0.000130,2.961421e-04,...,0.000191,0.014283,0.000716,2.952401e-04,0.477404,0.017661,0.005237,0.003527,0.020041,0.000296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nexus-mutual,0.017720,0.017252,0.076663,0.021673,0.017661,0.016170,1.795732e-02,0.013423,0.017791,1.795763e-02,...,0.017853,0.003379,0.016946,1.795673e-02,0.459743,0.000000,0.012425,0.014135,0.002379,0.017957
juno,0.005296,0.004828,0.089088,0.034098,0.005237,0.003746,5.532731e-03,0.000999,0.005367,5.533044e-03,...,0.005428,0.009046,0.004521,5.532142e-03,0.472167,0.012425,0.000000,0.001710,0.014804,0.005533
okb,0.003586,0.003118,0.090798,0.035808,0.003527,0.002036,3.822801e-03,0.000711,0.003657,3.823114e-03,...,0.003718,0.010756,0.002811,3.822212e-03,0.473877,0.014135,0.001710,0.000000,0.016514,0.003823


In [5]:
#Test output of the weighted average similarity matrix
coin_df_weighted_avg

,iota,anchor-protocol,compound,bitcoin-sv,drep,moonbeam,usd-coin,chainlink,basic-attention-token,bittorrent,...,gala,bitcoin-gold,render-token,unfoldu-group-coin-(new),maker,nexus-mutual,juno,okb,avalanche,compound-usd-coin
Name,,,,,,,,,,,,,,,,,,,,,
iota,0.000000,1.585413,1.132772,1.134665,1.367762,1.129967,1.104334,1.054114,1.112964,1.188312,...,1.386424,1.174473,1.353307,1.196091,1.289819,1.104241,1.190276,1.091185,1.268348,1.254620
anchor-protocol,1.585413,0.000000,1.710326,1.658395,1.704004,1.634555,1.685107,1.672283,1.691185,1.661528,...,1.725330,1.653308,1.680531,1.604430,1.834426,1.635281,1.675627,1.577806,1.700303,1.686370
compound,1.132772,1.710326,0.000000,1.168332,1.492708,1.246704,1.141547,1.113999,1.113956,1.267199,...,1.509982,1.219497,1.432081,1.243953,1.091590,1.242307,1.249684,1.196786,1.281972,1.171455
bitcoin-sv,1.134665,1.658395,1.168332,0.000000,1.441280,1.215464,1.158710,1.156518,1.165853,1.275123,...,1.478520,1.061680,1.395479,1.258499,1.335629,1.243452,1.270657,1.172152,1.296790,1.331453
drep,1.367762,1.704004,1.492708,1.441280,0.000000,1.405938,1.434443,1.387245,1.463192,1.485330,...,1.499512,1.420867,1.504448,1.422156,1.679634,1.430872,1.423011,1.384071,1.428498,1.510186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nexus-mutual,1.104241,1.635281,1.242307,1.243452,1.430872,1.237355,1.237361,1.193921,1.223174,1.235702,...,1.442503,1.258667,1.389077,1.194801,1.429435,0.000000,1.265483,1.125950,1.293109,1.360660
juno,1.190276,1.675627,1.249684,1.270657,1.423011,1.183598,1.208807,1.120020,1.254911,1.313437,...,1.545719,1.259777,1.456823,1.275749,1.419782,1.265483,0.000000,1.195276,1.277488,1.371807
okb,1.091185,1.577806,1.196786,1.172152,1.384071,1.008298,1.145252,1.150213,1.178193,1.128738,...,1.411269,1.123027,1.339289,1.131788,1.352987,1.125950,1.195276,0.000000,1.201990,1.259666


In [6]:
#Test output of the processed coin info dataset.
processed_coin_df

,Name,Volatility,Description
0,iota,0.388529,IOTA (IOTA or MIOTA) is a cryptocurrency token...
1,anchor-protocol,1.155277,Anchor Protocol is a yield stable and attracti...
2,compound,155.017778,COMP is an ERC-20 token built on the Ethereum ...
3,bitcoin-sv,64.927187,Bitcoin SV is a cryptocurrency that was create...
4,drep,0.485170,DREPis committed to building a performance-ori...
...,...,...,...
136,nexus-mutual,29.419982,Nexus Mutal is a new concept that is built as ...
137,juno,9.064785,Jun is an open source platform for interoperab...
138,okb,6.263407,OKB is a utility token issued by the OKB found...
139,avalanche,33.317865,Avalanche is an open-source platform for launc...


## Read In Target Coin Name

In [7]:
with open("coin_data.json") as f :
    name = json.load(f) #Read the JSON file which stores the coin name.

target_coin_name = name["coinName"] #Extract the coin name from the JSON file.
print(target_coin_name)

usd-coin


The target coin for which we wish to determine similar coins to is stored in the coin_data.json file. The reason for this is to ensure the security and maintainability of the developed codebase. For example, if this project were handed off to a client, it would be preferable for them to modify the desired coin name within the JSON file instead of changing a hardcoded coin name within this script, potentially changing something in the code unintentionally which could cause it to break.

## Determine Sorted Similarities to Target Coin

In [8]:
#Function to sort coins in ascending order in terms of their similarity to a target coin.
def output_similarity(df, coin_name) :
    scores = list(enumerate(df[coin_name]))
    sorted_scores = sorted(scores, key = lambda x :x[1], reverse = False)
    sorted_scores = sorted_scores[1:] #Removes first measure in sorted coin list. This is needed to remove self similarity. For example we do not care if a coin is exactly the same as itself.
    return sorted_scores

#Function that returns the number of coins in the sorted list. It is used when determing the number of similar coins to recommend through the kneed method.
def num_coins(score_list) :
    num_coins = []
    for i in range(1, len(score_list) + 1) :
        num_coins.append(i)
    return num_coins

#Function that returns sorted list of coins in terms of their similarity to a target coin.
def coin_data(df, coin_name) :
    coins = list(df[coin_name])
    sorted_coins = sorted(coins)
    sorted_coins = sorted_coins[1:]
    return sorted_coins

#Function that outputs the n-number of similar coins to the target coin.
def num_similar_coins(df, score_list, kneedle_point, coin_name, method) :
    j = 0
    print("The top " + str(kneedle_point) + " coins to " + coin_name + " by method of " + method + " are: ")
    for item in score_list :
        similar_coin_name = processed_coin_df["Name"][item[0]]
        print(similar_coin_name) #Printing similar coin names.
        j +=1
        if j > int(kneedle_point) - 1 : #Stop after n coins have been printed.
            break

In [9]:
wmd_coin_similarity = output_similarity(coin_df_wmd, target_coin_name) #Get sorted similarity of coins in comparison to the target coin.
wmd_num_coins = num_coins(wmd_coin_similarity) #Get the number of sorted coins.
wmd_coin_data = coin_data(coin_df_wmd, target_coin_name) #Gets sorted coin list which is used to determine knee point.
kneedle_wmd = KneeLocator(wmd_num_coins, wmd_coin_data) #Locate the knee point of the data. This will act as the threshold of the number of similar coins to output.
kneedle_point_wmd = kneedle_wmd.knee #Save the knee point in a variable.
method = "description similarity through Word Mover's Distance" #Message to be used to display how the similar coins were clustered.
num_similar_coins(processed_coin_df, wmd_coin_similarity, kneedle_point_wmd, target_coin_name, method) #Use the knee point to display the top n similar coins.



The top 8 coins to usd-coin by method of description similarity through Word Mover's Distance are: 
tether
binance-coin
maker
ethereum
tezos
cardano
bitcoin
stellar


In [10]:
ed_coin_similarity = output_similarity(coin_df_ed, target_coin_name) #Get sorted similarity of coins in comparison to the target coin.
ed_num_coins = num_coins(ed_coin_similarity)  #Get the number of sorted coins.
ed_coin_data = coin_data(coin_df_ed, target_coin_name) #Gets sorted coin list which is used to determine knee point.
kneedle_ed = KneeLocator(ed_num_coins, ed_coin_data) #Locate the knee point of the data. This will act as the threshold of the number of similar coins to output.
kneedle_point_ed = kneedle_ed.knee #Save the knee point in a variable.
method = "volatility similarity through Euclidean Distance" #Message to be used to display how the similar coins were clustered.
num_similar_coins(processed_coin_df, ed_coin_similarity, kneedle_point_ed, target_coin_name, method) #Use the knee point to display the top n similar coins.

The top 1 coins to usd-coin by method of volatility similarity through Euclidean Distance are: 
compound-usd-coin


In [11]:
weighted_average_coin_similarity = output_similarity(coin_df_weighted_avg, target_coin_name) #Get sorted similarity of coins in comparison to the target coin.
weighted_average_num_coins = num_coins(weighted_average_coin_similarity) #Get the number of sorted coins.
weighted_average_coin_data = coin_data(coin_df_weighted_avg, target_coin_name) #Gets sorted coin list which is used to determine knee point.
kneedle_weighted_average = KneeLocator(weighted_average_num_coins, weighted_average_coin_data) #Locate the knee point of the data. This will act as the threshold of the number of similar coins to output.
kneedle_point_weighted_average = kneedle_weighted_average.knee #Save the knee point in a variable.
method = "a weighted average of volatility and description similarity" #Message to be used to display how the similar coins were clustered.
num_similar_coins(processed_coin_df, weighted_average_coin_similarity, kneedle_point_weighted_average, target_coin_name, method)  #Use the knee point to display the top n similar coins.

The top 2 coins to usd-coin by method of a weighted average of volatility and description similarity are: 
tether
binance-coin


Based on the above results, a preliminary result can be drawn regarding the accuracy of the clustering mechanisms utilized. The semantic similarity mechanism (Word Mover's Distance) is shown to be considerable better than the volatility (measured through Euclidean Distance) and average of both of these metrics. An explanation could be that volatility is always changing in an unpredictable way which could potentially make it hard to predict similar coins.